# Data Sources
## Data files saved in 'Data' folder

GDP by MSA – 2001 -2017 data ( MAGDP files)
Population, Per capita personal income  2001-2017 data by counties (CAIN30 file) (2001-2017)
2003- 2019 Airportwise passengers/flights data at https://www.transtats.bts.gov/Data_Elements.aspx?Data=1

Airquality data by counties
https://aqs.epa.gov/aqsweb/airdata/download_files.html#Annual (1980-2019)


In [1]:
import pandas as pd

# Collecting Air Quality indicators

In [2]:
import glob
import os

path = 'Data/AirQuality' # use correct path for Airquality folder
all_files = glob.glob(os.path.join(path, "*.zip"))

df_AirQ = pd.concat((pd.read_csv(f) for f in all_files))

In [3]:
df_AirQ.columns

Index(['State', 'County', 'Year', 'Days with AQI', 'Good Days',
       'Moderate Days', 'Unhealthy for Sensitive Groups Days',
       'Unhealthy Days', 'Very Unhealthy Days', 'Hazardous Days', 'Max AQI',
       '90th Percentile AQI', 'Median AQI', 'Days CO', 'Days NO2',
       'Days Ozone', 'Days SO2', 'Days PM2.5', 'Days PM10'],
      dtype='object')

In [4]:
df_AirQ = df_AirQ[['Year', 'State', 'County', 'Days with AQI','Good Days', 'Max AQI','90th Percentile AQI', 'Median AQI'] ]  

In [5]:
df_AirQ.head()

,Year,State,County,Days with AQI,Good Days,Max AQI,90th Percentile AQI,Median AQI
0,2008,Alabama,Baldwin,265,210,133,68,37
1,2008,Alabama,Clay,110,70,93,61,43
2,2008,Alabama,Colbert,277,201,97,64,43
3,2008,Alabama,DeKalb,110,63,87,64,48
4,2008,Alabama,Elmore,245,180,105,74,42


In [6]:
df_AirQ.Year.unique()

array([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018],
      dtype=int64)

In [7]:
df_AirQ.State.unique(), df_AirQ.County.unique()[1:5]

(array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
        'Colorado', 'Connecticut', 'Country Of Mexico', 'Delaware',
        'District Of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
        'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
        'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
        'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
        'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
        'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
        'Pennsylvania', 'Puerto Rico', 'Rhode Island', 'South Carolina',
        'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
        'Virgin Islands', 'Virginia', 'Washington', 'West Virginia',
        'Wisconsin', 'Wyoming'], dtype=object),
 array(['Clay', 'Colbert', 'DeKalb', 'Elmore'], dtype=object))

In [8]:
df_Air = df_AirQ.loc[(df_AirQ.State == 'California') | (df_AirQ.State == 'New York') | (df_AirQ.State == 'New Jersey')]
df_Air.shape

(1112, 8)

In [9]:
LA_counties = ['Los Angeles', 'Orange']
SF_counties =['Alameda', 'Contra Costa', 'Marin', 'San Francisco', 'San Mateo']
NY_counties = ['Bergen', 'Essex', 'Hudson', 'Hunterdon', 'Middlesex', 'Monmouth',
              'Morris', 'Ocean', 'Passic', 'Somerset', 'Sussex', 'Union', 'Bronx', 'Dutchess',
              'Kings', 'Nasseu', 'New York', 'Orange', 'Putnam', 'Queens', 'Richmond',
              'Rockland', 'Suffolk', 'Westcher']

In [10]:
df_Air = df_Air[df_Air['County'].isin(LA_counties+ SF_counties+ NY_counties)]
df_Air.County.unique()

array(['Alameda', 'Contra Costa', 'Kings', 'Los Angeles', 'Marin',
       'Orange', 'San Francisco', 'San Mateo', 'Bergen', 'Essex',
       'Hudson', 'Hunterdon', 'Middlesex', 'Monmouth', 'Morris', 'Ocean',
       'Union', 'Bronx', 'Dutchess', 'New York', 'Putnam', 'Queens',
       'Richmond', 'Suffolk', 'Rockland'], dtype=object)

In [11]:
import numpy as np
conditions = [(df_Air['County'].isin(LA_counties)), (df_Air['County'].isin(SF_counties)), (df_Air['County'].isin(NY_counties))]
choices = ['LA_MSA', 'SF_MSA', 'NY_MSA']
df_Air['MSA'] =  np.select(conditions, choices)

In [12]:
df_Air.head()

,Year,State,County,Days with AQI,Good Days,Max AQI,90th Percentile AQI,Median AQI,MSA
56,2008,California,Alameda,366,182,205,83,51,SF_MSA
61,2008,California,Contra Costa,366,266,161,77,42,SF_MSA
70,2008,California,Kings,366,68,220,143,80,NY_MSA
72,2008,California,Los Angeles,366,31,227,172,84,LA_MSA
74,2008,California,Marin,363,347,97,44,31,SF_MSA


In [13]:
df_Air['Good_Days_Percent'] = 100* df_Air['Good Days']/df_Air['Days with AQI']
df_Air.columns

Index(['Year', 'State', 'County', 'Days with AQI', 'Good Days', 'Max AQI',
       '90th Percentile AQI', 'Median AQI', 'MSA', 'Good_Days_Percent'],
      dtype='object')

In [14]:
df_AirQuality = df_Air.pivot_table(index = ['MSA', 'Year'], values =['Median AQI', 'Good_Days_Percent'], aggfunc = np.mean)
df_AirQuality.shape
df_AirQuality.columns = [['Median_AQI','Good_Days_Percent']]

In [15]:
df_AirQuality.head()

Median_AQI Good_Days_Percent
MSA    Year                             
LA_MSA 2008  35.974499         63.333333
       2009  39.726027         58.333333
       2010  38.492019         57.333333
       2011  36.255708         59.000000
       2012  42.531876         58.000000

# Population and Per Capita Income

In [16]:
df_income_pop = pd.read_excel('Data/Population_Income/CAINC30__ALL_STATES_1969_2017.xlsx')

In [17]:
#Population (persons) 3/, Per capita personal income 4/


df_income = df_income_pop[df_income_pop.LineCode == 110]
df_population = df_income_pop[df_income_pop.LineCode == 100]
df_income.shape, df_population.shape

((3198, 57), (3198, 57))

In [18]:
df_income.tail(2)

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,1969,1970,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
99084,"""97000""",Rocky Mountain,7,CAINC30,110.0,...,Per capita personal income 4/,Dollars,3602,3956,...,38567,36273,37054,39569,41677,42701,45307,47053,47505,49265
99115,"""98000""",Far West,8,CAINC30,110.0,...,Per capita personal income 4/,Dollars,4570,4855,...,43214,41298,42636,44966,47451,47819,50787,53911,55550,57748


In [19]:
df_income[df_income.GeoName.str.contains("Orange, CA")]

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,1969,1970,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
7603,"""06059""","Orange, CA",8,CAINC30,110.0,...,Per capita personal income 4/,Dollars,4821,5074,...,50028,46813,49740,51469,55296,54594,57110,61178,62763,65400


In [20]:
LA_counties1 = ['Los Angeles, CA', 'Orange, CA']
SF_counties1 =['Alameda, CA', 'Contra Costa, CA', 'Marin, CA', 'San Francisco, CA', 'San Mateo, CA']
NY_counties1 = ['Bergen, NJ', 'Essex, NJ', 'Hudson, NJ', 'Hunterdon, NJ', 'Middlesex, NJ', 'Monmouth, NJ',
              'Morris, NJ', 'Ocean, NJ', 'Passic, NJ', 'Somerset, NJ', 'Sussex, NJ', 'Union, NY', 'Bronx, NY', 'Dutchess, NY',
              'Kings, NY', 'Nasseu, NY', 'New York, NY', 'Orange, NY', 'Putnam, NY', 'Queens, NY', 'Richmond, NY',
              'Rockland, NY', 'Suffolk, NY', 'Westcher, NY']


In [21]:
df_income = df_income[['GeoName',2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]]
df_population = df_population[['GeoName', 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]]
df_population.tail()

,GeoName,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
98990,Plains,20248815,20392583,20535719,20638502,20749482,20872827,20984812,21082599,21179519,21294107
99021,Southeast,77057840,77814936,78575213,79260002,79953925,80585268,81304675,82104601,82935602,83715076
99052,Southwest,36269039,36899289,37472786,37972888,38521426,39030404,39619086,40243759,40820143,41339800
99083,Rocky Mountain,10609537,10793888,10949084,11079489,11215355,11366966,11514638,11689105,11878018,12055738
99114,Far West,51608614,52167532,52686778,53176406,53679832,54161802,54695081,55249155,55772858,56250544


In [22]:
df_income = df_income[df_income['GeoName'].isin(LA_counties1+ SF_counties1+ NY_counties1)]
df_population = df_population[df_population['GeoName'].isin(LA_counties1+ SF_counties1+ NY_counties1)]


In [23]:
numeric = [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
for year in numeric:
    df_income[year] = df_income[year].astype(int)
    df_population[year] = df_population[year].astype(int)
    
df_income.dtypes

GeoName    object
2008        int32
2009        int32
2010        int32
2011        int32
2012        int32
2013        int32
2014        int32
2015        int32
2016        int32
2017        int32
dtype: object

In [24]:
df_income.head()

,GeoName,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
6704,"Alameda, CA",48943,46330,48118,51247,53323,55378,58984,63809,67356,71282
6890,"Contra Costa, CA",58906,54242,54030,57681,61878,61435,64056,69195,72195,76527
7262,"Los Angeles, CA",43431,41869,43569,46439,49459,49010,52130,55366,56851,58419
7324,"Marin, CA",90711,82249,84002,90963,97274,98203,106548,114455,117552,124552
7603,"Orange, CA",50028,46813,49740,51469,55296,54594,57110,61178,62763,65400


In [25]:
import numpy as np
conditions1 = [(df_income['GeoName'].isin(LA_counties1)), 
              (df_income['GeoName'].isin(SF_counties1)), 
              (df_income['GeoName'].isin(NY_counties1))]

conditions2 = [(df_population['GeoName'].isin(LA_counties1)), 
              (df_population['GeoName'].isin(SF_counties1)), 
              (df_population['GeoName'].isin(NY_counties1))]

choices = ['LA_MSA', 'SF_MSA', 'NY_MSA']
df_income['MSA'] =  np.select(conditions1, choices)
df_population['MSA'] =  np.select(conditions2, choices)

In [26]:
df_income.head(2)

,GeoName,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,MSA
6704,"Alameda, CA",48943,46330,48118,51247,53323,55378,58984,63809,67356,71282,SF_MSA
6890,"Contra Costa, CA",58906,54242,54030,57681,61878,61435,64056,69195,72195,76527,SF_MSA


In [27]:
dfi =df_income.pivot_table(index = ['MSA'], values = [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017], aggfunc = np.median )
dfp =df_population.pivot_table(index = ['MSA'], values = [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017], aggfunc = np.sum )

In [28]:
df_population.head()

,GeoName,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,MSA
6703,"Alameda, CA",1477208,1498539,1513402,1532215,1556648,1582936,1611572,1637141,1653236,1663190,SF_MSA
6889,"Contra Costa, CA",1023344,1037890,1052799,1066182,1079093,1095310,1110971,1126027,1138645,1147439,SF_MSA
7261,"Los Angeles, CA",9735147,9787400,9824490,9885998,9956152,10018604,10072695,10123248,10150558,10163507,LA_MSA
7323,"Marin, CA",248398,250862,252940,255505,256381,258856,261007,261718,261532,260955,SF_MSA
7602,"Orange, CA",2957593,2987177,3017116,3053465,3085386,3113649,3136750,3160576,3177703,3190400,LA_MSA


In [29]:
dfi = dfi.T
dfp = dfp.T

In [30]:
dfi = dfi.reset_index()
dfp = dfp.reset_index()
dfi['MSA'] = 'LA_MSA'
dfp['MSA'] = 'LA_MSA'
dfi.head()

MSA,index,LA_MSA,NY_MSA,SF_MSA,MSA
0,2008,46729.5,48880.5,75468.0,LA_MSA
1,2009,44341.0,47343.0,71694.0,LA_MSA
2,2010,46654.5,48082.0,73739.0,LA_MSA
3,2011,48954.0,50679.0,79872.0,LA_MSA
4,2012,52377.5,52804.0,87986.0,LA_MSA


In [31]:
dfi1 = dfi[['index', 'LA_MSA']]
dfp1 = dfp[['index', 'LA_MSA']]
dfi2 = dfi[['index', 'SF_MSA']]
dfp2 = dfp[['index', 'SF_MSA']]
dfi3 = dfi[['index', 'NY_MSA']]
dfp3 = dfp[['index', 'NY_MSA']]
dfi1.columns  = ['Year', 'Per_Capita_Income']
dfi2.columns  = ['Year', 'Per_Capita_Income']
dfi3.columns  = ['Year', 'Per_Capita_Income']
dfp1.columns  = ['Year', 'Population']
dfp2.columns  = ['Year', 'Population']
dfp3.columns  = ['Year', 'Population']
dfi1['MSA'] = 'LA_MSA'
dfi2['MSA'] = 'SF_MSA'
dfi3['MSA'] = 'NY_MSA'
dfp1['MSA'] = 'LA_MSA'
dfp2['MSA'] = 'SF_MSA'
dfp3['MSA'] = 'NY_MSA'

C:\Users\avise\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\avise\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\avise\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable

In [32]:
dfi = pd.concat([dfi1, dfi2, dfi3], axis =0)
dfi =dfi[['MSA', 'Year', 'Per_Capita_Income']]
dfp = pd.concat([dfp1, dfp2, dfp3], axis =0)
dfp =dfp[['MSA', 'Year', 'Population']]
dfi = dfi.set_index(['MSA', 'Year'], drop = True)
dfp = dfp.set_index(['MSA', 'Year'], drop = True)
dfp.head()

Population
MSA    Year            
LA_MSA 2008    12692740
       2009    12774577
       2010    12841606
       2011    12939463
       2012    13041538

# Airtraffic - Passengers boarding

In [33]:
import pandas as pd
import glob
import os

path_la = 'Data/Airtraffic/LA' 
all_files_la = glob.glob(os.path.join(path_la, "*.csv"))


path_sf = 'Data/Airtraffic/SF' 
all_files_sf = glob.glob(os.path.join(path_sf, "*.csv"))

path_ny = 'Data/Airtraffic/NY' 
all_files_ny = glob.glob(os.path.join(path_ny, "*.csv"))

df1la = pd.read_csv(all_files_la[0])
df2la = pd.read_csv(all_files_la[1])
df3la = pd.read_csv(all_files_la[2])

df1sf = pd.read_csv(all_files_sf[0])
df2sf = pd.read_csv(all_files_sf[1])
df3sf = pd.read_csv(all_files_sf[2])

df1ny = pd.read_csv(all_files_ny[0])
df2ny = pd.read_csv(all_files_ny[1])
df3ny = pd.read_csv(all_files_ny[2])



In [34]:
def transform(df):
    df = df.rename(columns=df.iloc[0])
    df.drop(0, inplace=True)
    df.columns = ['Year', 'Month', 'DOMESTIC', 'INTERNATIONAL', 'TOTAL']
    df = df[['Year', 'Month', 'TOTAL']]
    df['Passengers'] = df.TOTAL.str.replace(',', '').astype(float)
    df.drop('TOTAL', axis = 1, inplace = True)
    df = df[(df.Month == 'TOTAL') & (df.Year != '2019') & (df.Year != '2002') 
          & (df.Year != '2003')& (df.Year != '2004')& (df.Year != '2005')& 
          (df.Year != '2006')& (df.Year != '2007') ]
    
    return df

In [35]:
df1la = transform(df1la)
df2la = transform(df2la)
df3la = transform(df3la)
df_atraf_la = df1la
df_atraf_la['Passengers'] = df1la.Passengers+df2la.Passengers+df3la.Passengers
df_atraf_la['MSA'] = 'LA_MSA'
df_atraf_la.head(20)

df1sf = transform(df1sf)
df2sf = transform(df2sf)
df3sf = transform(df3sf)
df_atraf_sf = df1sf
df_atraf_sf['Passengers'] = df1sf.Passengers+df2sf.Passengers+df3sf.Passengers
df_atraf_sf['MSA'] = 'SF_MSA'
df_atraf_sf.head(20)

df1ny = transform(df1ny)
df2ny = transform(df2ny)
df3ny = transform(df3ny)
df_atraf_ny = df1ny
df_atraf_ny['Passengers'] = df1ny.Passengers+df2ny.Passengers+df3ny.Passengers
df_atraf_ny['MSA'] = 'NY_MSA'
df_atraf_ny.head(20)

,Year,Month,Passengers,MSA
82,2008,TOTAL,52866278.0,NY_MSA
95,2009,TOTAL,50476621.0,NY_MSA
108,2010,TOTAL,51424463.0,NY_MSA
121,2011,TOTAL,52355837.0,NY_MSA
134,2012,TOTAL,54229063.0,NY_MSA
147,2013,TOTAL,55995808.0,NY_MSA
160,2014,TOTAL,57497612.0,NY_MSA
173,2015,TOTAL,60916294.0,NY_MSA
186,2016,TOTAL,63798888.0,NY_MSA
199,2017,TOTAL,65634288.0,NY_MSA


In [36]:
df_Airtraffic = pd.concat([df_atraf_la, df_atraf_sf, df_atraf_ny])
df_Airtraffic['Year'] = df_Airtraffic['Year'].astype(int)
df_Airtraffic = df_Airtraffic.set_index(['MSA', 'Year'], drop = True)
df_Airtraffic.drop('Month', axis = 1, inplace = True)
df_Airtraffic

Passengers
MSA    Year            
LA_MSA 2008  36273289.0
       2009  34179598.0
       2010  35475341.0
       2011  37011731.0
       2012  37790767.0
       2013  38908822.0
       2014  40931688.0
       2015  43464589.0
       2016  46920468.0
       2017  48525247.0
       2018  50363107.0
SF_MSA 2008  28479144.0
       2009  27161820.0
       2010  28032599.0
       2011  28661968.0
       2012  30250725.0
       2013  30735736.0
       2014  32411808.0
       2015  34533303.0
       2016  36917676.0
       2017  39404268.0
       2018  41493480.0
NY_MSA 2008  52866278.0
       2009  50476621.0
       2010  51424463.0
       2011  52355837.0
       2012  54229063.0
       2013  55995808.0
       2014  57497612.0
       2015  60916294.0
       2016  63798888.0
       2017  65634288.0
       2018  68431481.0

# GDP by MSA

In [37]:
df_gdp = pd.read_csv('Data/GDP_MSA/MAGDP2_2001_2017_ALL_AREAS.csv')

In [38]:
df_gdp.GeoName.unique()

array(['United States (Metropolitan Portion)',
       'Abilene, TX (Metropolitan Statistical Area)',
       'Akron, OH (Metropolitan Statistical Area)',
       'Albany, GA (Metropolitan Statistical Area)',
       'Albany, OR (Metropolitan Statistical Area)',
       'Albany-Schenectady-Troy, NY (Metropolitan Statistical Area)',
       'Albuquerque, NM (Metropolitan Statistical Area)',
       'Alexandria, LA (Metropolitan Statistical Area)',
       'Allentown-Bethlehem-Easton, PA-NJ (Metropolitan Statistical Area)',
       'Altoona, PA (Metropolitan Statistical Area)',
       'Amarillo, TX (Metropolitan Statistical Area)',
       'Ames, IA (Metropolitan Statistical Area)',
       'Anchorage, AK (Metropolitan Statistical Area)',
       'Ann Arbor, MI (Metropolitan Statistical Area)',
       'Anniston-Oxford-Jacksonville, AL (Metropolitan Statistical Area)',
       'Appleton, WI (Metropolitan Statistical Area)',
       'Asheville, NC (Metropolitan Statistical Area)',
       'Athens-Clarke 

In [39]:
LA_MSA = 'Los Angeles-Long Beach-Anaheim, CA (Metropolitan Statistical Area)'
SF_MSA = 'San Francisco-Oakland-Hayward, CA (Metropolitan Statistical Area)'
NY_MSA = 'New York-Newark-Jersey City, NY-NJ-PA (Metropolitan Statistical Area)'

df_gdp = df_gdp[df_gdp.Description == 'All industry total']
df_gdp.columns

Index(['GeoFIPS', 'GeoName', 'Region', 'TableName', 'ComponentName', 'Unit',
       'IndustryId', 'IndustryClassification', 'Description', '2001', '2002',
       '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017'],
      dtype='object')

In [40]:
select_columns = ['GeoName', '2008', '2009', '2010', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017']
df_gdp = df_gdp[select_columns]
df_gdp.shape

(384, 11)

In [41]:
df_gdp = df_gdp[df_gdp.GeoName.isin([LA_MSA, SF_MSA, NY_MSA])]
df_gdp.head()

,GeoName,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
18357,"Los Angeles-Long Beach-Anaheim, CA (Metropolit...",772747.1,741630.0,763975.9,782565.1,820862.6,852034.2,901979.9,967100.1,996431.8,1043735.1
21750,"New York-Newark-Jersey City, NY-NJ-PA (Metropo...",1257147.7,1276469.8,1338450.7,1363858.9,1439232.6,1477043.4,1542763.2,1618365.7,1662671.3,1717712.2
27057,"San Francisco-Oakland-Hayward, CA (Metropolita...",353338.8,331326.0,330153.6,340757.4,366151.4,385451.5,413026.5,445123.5,475416.6,500710.1


In [42]:
import numpy as np
conditions3 = [(df_gdp['GeoName'] == LA_MSA), 
              (df_gdp['GeoName'] == SF_MSA), 
              (df_gdp['GeoName'] == NY_MSA)]


choices = ['LA_MSA', 'SF_MSA', 'NY_MSA']
df_gdp['MSA'] =  np.select(conditions3, choices)
df_gdp.drop('GeoName', axis = 1, inplace = True)

df_gdp

,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,MSA
18357,772747.1,741630.0,763975.9,782565.1,820862.6,852034.2,901979.9,967100.1,996431.8,1043735.1,LA_MSA
21750,1257147.7,1276469.8,1338450.7,1363858.9,1439232.6,1477043.4,1542763.2,1618365.7,1662671.3,1717712.2,NY_MSA
27057,353338.8,331326.0,330153.6,340757.4,366151.4,385451.5,413026.5,445123.5,475416.6,500710.1,SF_MSA


In [43]:
df_GDP= df_gdp

In [44]:
df_GDP = df_GDP.melt(id_vars=['MSA'], 
        var_name="Year", 
        value_name="GDP")
df_GDP['Year']=df_GDP['Year'].astype(int)
df_GDP['GDP']=df_GDP['GDP'].astype(float)
df_GDP.set_index(['MSA', 'Year'], inplace = True, drop = True)
df_GDP.head()

,,GDP
MSA,Year,
LA_MSA,2008,772747.1
NY_MSA,2008,1257147.7
SF_MSA,2008,353338.8
LA_MSA,2009,741630.0
NY_MSA,2009,1276469.8


In [45]:
df_GDP.shape, df_Airtraffic.shape, dfp.shape, dfi.shape, df_AirQuality.shape

((30, 1), (33, 1), (30, 1), (30, 1), (33, 2))

In [46]:
list_df = [df_GDP, dfi, dfp,  df_Airtraffic,df_AirQuality]
df_final = df_GDP
for frame in list_df[1:]:
    df_final = pd.merge(df_final, frame, left_index = True, right_index = True, how = 'outer')
df_final.head()

GDP  Per_Capita_Income  Population  Passengers  \
MSA    Year                                                        
LA_MSA 2008  772747.1            46729.5  12692740.0  36273289.0   
       2009  741630.0            44341.0  12774577.0  34179598.0   
       2010  763975.9            46654.5  12841606.0  35475341.0   
       2011  782565.1            48954.0  12939463.0  37011731.0   
       2012  820862.6            52377.5  13041538.0  37790767.0   

             (Median_AQI,)  (Good_Days_Percent,)  
MSA    Year                                       
LA_MSA 2008      35.974499             63.333333  
       2009      39.726027             58.333333  
       2010      38.492019             57.333333  
       2011      36.255708             59.000000  
       2012      42.531876             58.000000

In [47]:
df_final.reset_index(inplace = True)

In [48]:
df_final.columns

Index([                 'MSA',                 'Year',                  'GDP',
          'Per_Capita_Income',           'Population',           'Passengers',
              ('Median_AQI',), ('Good_Days_Percent',)],
      dtype='object')

In [49]:
df_final['GDP'] = df_final['GDP']/1000
df_final['Passengers'] = df_final['Passengers']/1000000
df_final['Population'] = df_final['Population']/1000000

df_final.columns = ['MSA', 'Year', 'GDP_bn','Per_Capita_Income','Population_mn','Passengers_mn','Median_AQI', 'Good_Days_Percent']

In [50]:
df_final = round(df_final,2)
df_final.head()

,MSA,Year,GDP_bn,Per_Capita_Income,Population_mn,Passengers_mn,Median_AQI,Good_Days_Percent
0,LA_MSA,2008,772.75,46729.5,12.69,36.27,35.97,63.33
1,LA_MSA,2009,741.63,44341.0,12.77,34.18,39.73,58.33
2,LA_MSA,2010,763.98,46654.5,12.84,35.48,38.49,57.33
3,LA_MSA,2011,782.57,48954.0,12.94,37.01,36.26,59.00
4,LA_MSA,2012,820.86,52377.5,13.04,37.79,42.53,58.00


In [56]:
df_final.dtypes

MSA                   object
Year                   int64
GDP_bn               float64
Per_Capita_Income    float64
Population_mn        float64
Passengers_mn        float64
Median_AQI           float64
Good_Days_Percent    float64
dtype: object

In [57]:
zillow_hv.dtypes

NameError: name 'zillow_hv' is not defined

In [59]:
#read from zillow file
df_temp = pd.read_csv("Data/housing_final.csv")
df_temp.dtypes
# df_all = pd.merge(df_final, df_temp, left_index = True, right_index = True, how = 'outer')

MSA                        object
Year                        int64
Zillow_home_value_index     int64
dtype: object

In [61]:
df_all.columns = ['MSA', 'Year', 'GDP_bn', 'Per_Capita_Income', 'Population_mn',
       'Passengers_mn', 'Median_AQI', 'Good_Days_Percent', 'MSA_y', 'Year_y',
       'Zillow_home_value_index']

In [63]:
df_final = df_all[['MSA', 'Year', 'GDP_bn', 'Per_Capita_Income', 'Population_mn',
       'Passengers_mn', 'Median_AQI', 'Good_Days_Percent','Zillow_home_value_index']]

In [64]:
df_final.to_csv('Data/final.csv', encoding = 'utf-8', index = False)

In [65]:
import pandas as pd
df_final = pd.read_csv('Data/final.csv')
df_final.dtypes

MSA                         object
Year                         int64
GDP_bn                     float64
Per_Capita_Income          float64
Population_mn              float64
Passengers_mn              float64
Median_AQI                 float64
Good_Days_Percent          float64
Zillow_home_value_index      int64
dtype: object

In [66]:
df_final

,MSA,Year,GDP_bn,Per_Capita_Income,Population_mn,Passengers_mn,Median_AQI,Good_Days_Percent,Zillow_home_value_index
0,LA_MSA,2008,772.75,46729.5,12.69,36.27,35.97,63.33,479416
1,LA_MSA,2009,741.63,44341.0,12.77,34.18,39.73,58.33,422158
2,LA_MSA,2010,763.98,46654.5,12.84,35.48,38.49,57.33,417525
3,LA_MSA,2011,782.57,48954.0,12.94,37.01,36.26,59.00,393241
4,LA_MSA,2012,820.86,52377.5,13.04,37.79,42.53,58.00,391958
5,LA_MSA,2013,852.03,51802.0,13.13,38.91,44.20,54.67,452550
6,LA_MSA,2014,901.98,54620.0,13.21,40.93,39.18,58.67,496241
7,LA_MSA,2015,967.10,58272.0,13.28,43.46,42.01,57.00,529591
8,LA_MSA,2016,996.43,59807.0,13.33,46.92,46.54,55.00,565766
9,LA_MSA,2017,1043.74,61909.5,13.35,48.53,47.40,55.00,605925


In [97]:
#fill Nulls with last year value + delta of previous 2 years
var_list = ['GDP_bn', 'Per_Capita_Income', 'Population_mn']
nan_index = [10,21,32]

In [101]:
for item in var_list:
    for id in nan_index:
        df_final[item][id] = df_final[item][id-1]+(df_final[item][id-1]-df_final[item][id-2])
        print(id)
df_final

C:\Users\avise\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


10
21
32
10
21
32
10
21
32


,MSA,Year,GDP_bn,Per_Capita_Income,Population_mn,Passengers_mn,Median_AQI,Good_Days_Percent,Zillow_home_value_index
0,LA_MSA,2008,772.75,46729.5,12.69,36.27,35.97,63.33,479416
1,LA_MSA,2009,741.63,44341.0,12.77,34.18,39.73,58.33,422158
2,LA_MSA,2010,763.98,46654.5,12.84,35.48,38.49,57.33,417525
3,LA_MSA,2011,782.57,48954.0,12.94,37.01,36.26,59.00,393241
4,LA_MSA,2012,820.86,52377.5,13.04,37.79,42.53,58.00,391958
5,LA_MSA,2013,852.03,51802.0,13.13,38.91,44.20,54.67,452550
6,LA_MSA,2014,901.98,54620.0,13.21,40.93,39.18,58.67,496241
7,LA_MSA,2015,967.10,58272.0,13.28,43.46,42.01,57.00,529591
8,LA_MSA,2016,996.43,59807.0,13.33,46.92,46.54,55.00,565766
9,LA_MSA,2017,1043.74,61909.5,13.35,48.53,47.40,55.00,605925


In [103]:
df_final.to_csv('Data/final.csv', encoding = 'utf-8', index = False)
import pandas as pd
df_final = pd.read_csv('Data/final.csv')
df_final.dtypes

MSA                         object
Year                         int64
GDP_bn                     float64
Per_Capita_Income          float64
Population_mn              float64
Passengers_mn              float64
Median_AQI                 float64
Good_Days_Percent          float64
Zillow_home_value_index      int64
dtype: object

# Creating SQLite database table using SQLAlchemy

In [104]:
import sqlalchemy
from sqlalchemy import *
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, ForeignKey, DateTime, Boolean, Numeric
# from sqlalchemy.orm import relationship, backref


In [105]:
engine = create_engine('sqlite:///dbase.db')

engine.execute('DROP TABLE IF EXISTS table1;')

Base = declarative_base()

metadata = MetaData(engine)


class Table1(Base):
    __tablename__ = 'table1'

    id = Column(Integer, primary_key=True)
    MSA = Column(String)
    Year= Column(Integer)
    GDP_bn = Column(Float)
    Per_Capita_Income = Column(Float)
    Population_mn = Column(Float)
    Passengers_mn = Column(Float)
    Median_AQI = Column(Float)
    Good_Days_Percent = Column(Float)
    Zillow_HVI = Column(Integer)

# bind this table definition/schema to sqlite database    
Base.metadata.create_all(engine)

# Populate records to the table from pandas dataframe
df_final.to_sql(con=engine, index_label='id', name=Table1.__tablename__, if_exists='replace')

In [106]:
engine.execute('select * from table1;').fetchall()

[(0, 'LA_MSA', 2008, 772.75, 46729.5, 12.69, 36.27, 35.97, 63.33, 479416),
 (1, 'LA_MSA', 2009, 741.63, 44341.0, 12.77, 34.18, 39.73, 58.33, 422158),
 (2, 'LA_MSA', 2010, 763.98, 46654.5, 12.84, 35.48, 38.49, 57.33, 417525),
 (3, 'LA_MSA', 2011, 782.57, 48954.0, 12.94, 37.01, 36.26, 59.0, 393241),
 (4, 'LA_MSA', 2012, 820.86, 52377.5, 13.04, 37.79, 42.53, 58.0, 391958),
 (5, 'LA_MSA', 2013, 852.03, 51802.0, 13.13, 38.91, 44.2, 54.67, 452550),
 (6, 'LA_MSA', 2014, 901.98, 54620.0, 13.21, 40.93, 39.18, 58.67, 496241),
 (7, 'LA_MSA', 2015, 967.1, 58272.0, 13.28, 43.46, 42.01, 57.0, 529591),
 (8, 'LA_MSA', 2016, 996.43, 59807.0, 13.33, 46.92, 46.54, 55.0, 565766),
 (9, 'LA_MSA', 2017, 1043.74, 61909.5, 13.35, 48.53, 47.4, 55.0, 605925),
 (10, 'LA_MSA', 2018, 1091.0500000000004, 64012.0, 13.37, 50.36, 50.78, 53.67, 644675),
 (11, 'NY_MSA', 2008, 1257.15, 48880.5, 16.0, 52.87, 60.86, 46.0, 413833),
 (12, 'NY_MSA', 2009, 1276.47, 47343.0, 16.1, 50.48, 70.82, 41.8, 376141),
 (13, 'NY_MSA', 201

In [107]:
Session = sessionmaker(bind = engine)
session = Session()

results = session.query(Table1.Passengers_mn).filter(Table1.MSA == 'LA_MSA').all()
for result in results:
    print(result)

(36.27,)
(34.18,)
(35.48,)
(37.01,)
(37.79,)
(38.91,)
(40.93,)
(43.46,)
(46.92,)
(48.53,)
(50.36,)


In [108]:
engine.execute('select count(*) from table1;').fetchall()

[(33,)]

In [72]:
!pwd


/c/Users/avise/desktop/bootcamp/git/Project2/Deepak
